# Sum Frame Sudoku Puzzle

Fill in numbers from 1 to 9 so that each row, column and 3x3 block contains each number exactly once.
Numbers in the outside frame equal the sum of the first three numbers in the corresponding row or column
in the given direction.

Find more examples [here](http://frame-sudoku.blogspot.com)

In [3]:
!pip install ortools

Defaulting to user installation because normal site-packages is not writeable


In [4]:
from ortools.sat.python import cp_model
from itertools import product

Sudoku puzzle from slides

In [5]:
top = [15, 18, 12, 11, 21, 13, 15, 17, 13]
right = [22, 8, 15, 22, 12, 11, 15, 13, 17]
bottom = [15, 9, 21, 10, 16, 19, 13, 15, 17]
left = [8, 15, 22, 11, 13, 21, 18, 19, 8]
n = 9

And another example (commented out)

In [ ]:
# top = [21, 12, 12, 13, 14, 18, 10, 19, 16]
# right = [20, 15, 10, 22, 8, 15, 17, 15, 13]
# bottom = [17, 9, 19, 18, 13, 14, 23, 15, 7]
# left = [12, 12, 21, 14, 14, 17, 14, 9, 22]

Create model

In [6]:
model = cp_model.CpModel()

In [7]:
# Type your model here ...
board = [[model.NewIntVar(1, n, f"({i},{j})") for j in range(n)] for i in range(n)]

In [8]:
# Each row / column contains only different values:
for i in range(n):
  model.AddAllDifferent([board[i][j] for j in range(n)]) # Rows
  model.AddAllDifferent([board[j][i] for j in range(n)]) # Columns
  
# Each 3x3 sub-grid contains only different values:
for i in range(3):
  for j in range(3):
    model.AddAllDifferent(
      [board[i * 3 + di][j * 3 + dj] for di in range(3) for dj in range(3)])
    
# 3 column values at the tops sum matches value in array row
for i in range(n):
  model.Add(cp_model.LinearExpr.Sum([board[i][0], board[i][1], board[i][2]]) == left[i])
  model.Add(cp_model.LinearExpr.Sum([board[i][6], board[i][7], board[i][8]]) == right[i])
  model.Add(cp_model.LinearExpr.Sum([board[0][i], board[1][i], board[2][i]]) == top[i])
  model.Add(cp_model.LinearExpr.Sum([board[6][i], board[7][i], board[8][i]]) == bottom[i])

Callback for solution printing (adapt if you do not use an n*n board)

In [9]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        
    def on_solution_callback(self):
        for i in range(len(self.__variables)):
            for j in range(len(self.__variables)):
                print(f"[{self.Value(self.__variables[i][j])}] ", end='')
            print("\n")
        print("\n\n")

Solve and print all solutions

In [10]:
solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, SolutionPrinter(board))

[1] [4] [3] [2] [8] [5] [9] [6] [7] 

[8] [5] [2] [6] [9] [7] [4] [3] [1] 

[6] [9] [7] [3] [4] [1] [2] [8] [5] 

[2] [3] [6] [7] [1] [4] [8] [5] [9] 

[4] [8] [1] [9] [5] [6] [3] [7] [2] 

[9] [7] [5] [8] [2] [3] [6] [1] [4] 

[7] [2] [9] [1] [3] [8] [5] [4] [6] 

[5] [6] [8] [4] [7] [2] [1] [9] [3] 

[3] [1] [4] [5] [6] [9] [7] [2] [8] 






In [11]:
print(f"Runtime:   {solver.WallTime()}ms")
print(f"Booleans:  {solver.NumBooleans()}")
print(f"Failures:  {solver.NumConflicts()}")
print(f"Branches:  {solver.NumBranches()}")

Runtime:   0.034972ms
Booleans:  0
Failures:  0
Branches:  0
